# Teste de Shapiro-Wilk

O teste de Shapiro-Wilk considera as seguintes hipóteses

## Importação de Bibliotecas

In [ ]:
pip install --upgrade openpyxl

In [2]:
import pandas as pd
import numpy as np
import random
import statistics
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

c:\Users\dorie\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\dorie\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
c:\Users\dorie\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Amostragem

*Escolha das amostras*

Distribuição Binomial

Distribuição Normal

Distribuição Poisson

Distribuição Lognormal

Amostra aleatória (sem especificação de distribuição)

In [61]:
# Definir a semente para garantir a reprodutibilidade
seeds = list(range(10))
numeros = list(range(7,19))

size_amostra = 10

# Listas para armazenar as amostras geradas
amostras_aleatorias = [] #amostra aleatória simples
amostras_lognormal = []
amostras_binomial = []
amostras_poisson = []
amostras_normal = []

# Gerar as amostras com seeds diferentes
for seed in seeds:
    random_state = np.random.RandomState(seed)
    amostra_aleatoria = random_state.choice(numeros, size_amostra)  # Amostra aleatória de 10 números
    amostra_lognormal = random_state.lognormal(25, 2, size_amostra)  # Distribuição log-normal com média 0 e desvio padrão 1
    amostra_binomial = random_state.binomial(10, 0.5, size_amostra)  # Distribuição binomial com 10 tentativas e probabilidade de sucesso 0.5
    amostra_poisson = random_state.poisson(2, size_amostra)  # Distribuição de Poisson com lambda = 2
    amostra_normal = random_state.normal(6, 1.5, size_amostra)  # Distribuição normal 
    
    amostras_aleatorias.append(amostra_aleatoria)
    amostras_lognormal.append(amostra_lognormal)
    amostras_binomial.append(amostra_binomial)
    amostras_poisson.append(amostra_poisson)
    amostras_normal.append(amostra_normal)




In [169]:
def transforma_lista_em_dataframe(amostras):
    df = pd.DataFrame(amostras)
    df.columns = ['X_i']
    df = df.sort_values(by='X_i').reset_index(drop=True)
    return df

df_aleatorias = transforma_lista_em_dataframe(amostras_aleatorias[0])

In [171]:
df_aleatorias

,X_i
0,7
1,9
2,10
3,10
4,10
5,12
6,12
7,14
8,16
9,18


## Calculando Dcalc

In [17]:
try:
    coeficient_table = pd.read_excel(open('Shapiro-Wilk-Tables.xlsx','rb'),sheet_name='SW Table',header=4,nrows=24)
except Exception as e:
    print("Ocorreu um erro ao ler o arquivo XLSX",e)

In [42]:
# Tratamento de valores NaN
coeficient_table = coeficient_table.fillna(0)

$W_{calc}=\frac{b^2}{\sum_{i=1}^{n}(X_i-\overline{X})^2}\quad para\quad i=1,...,n\\
b= \sum_{i=1}^{\frac{n}{2}}a_{i,n}(X_{(n-i-1)}-X_{(i)})$

In [110]:
df_aleatorias['X_i']

0     7
1     9
2    10
3    10
4    10
5    12
6    12
7    14
8    16
9    18
Name: X_i, dtype: int32

In [176]:
# Calculando b
n = 10
n_sobre_2 = int(n/2)
b = 0

for i in range(n_sobre_2):
    a = coeficient_table.iloc[i,n-1]
    dif = (df_aleatorias['X_i'][n-i-1] - df_aleatorias['X_i'][i])
    b += a*dif



In [177]:
b

9.797600000000001

In [175]:
media = amostra_aleatoria.mean()
media

13.6

In [180]:
numerador = np.power(b,2)

denominador = sum([(df_aleatorias['X_i'][i] - media)**2 for i in range(len(df_aleatorias))])

Wcalc = numerador/denominador

Wcalc


0.7163654161194031

## Valor crítico
n = 10 e a = 1% (0.01)

Olhando na tabela,
Wc = 0.781

In [179]:
Wc = 0.781
if Wcalc > Wc:
    print(f"{Wcalc:.3f} > {Wc:.3f}")
    print("A hipótese nula é aceita, logo os dados são normais")
else:
    print(f"{Wcalc:.3f} < {Wc:.3f}")
    print("A hipótese nula é rejeitada, logo os dados não são normais")

0.716 < 0.781
A hipótese nula é rejeitada, logo os dados não são normais


Utilizando a biblioteca Stats do Scipy

In [151]:
from scipy import stats

in_order = sorted(amostra_aleatoria)

res = stats.shapiro(in_order)
res

ShapiroResult(statistic=0.9445412158966064, pvalue=0.6045560836791992)

In [152]:
in_order

[8, 11, 12, 13, 13, 13, 15, 15, 18, 18]